## Equal-Weight S&P 500 Index Fund
Introduction & Library Imports
The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

In [106]:
# Import libraries

import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)

In [107]:
# Import list of stocks

stocks = pd.read_csv('sp_500_stocks.csv')
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
496,YUM
497,ZBH
498,ZBRA
499,ZION


In [108]:
# aquiring API token
from secret import IEX_CLOUD_API_TOKEN

In [109]:
# making out first API call
# market capitalisation for each stock
# price of each stock

symbol = "AAPL"
api_url = f"https://cloud.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}"
data = requests.get(api_url).json()
data

{'avgTotalVolume': 90297645,
 'calculationPrice': 'tops',
 'change': -4.12,
 'changePercent': -0.02759,
 'close': None,
 'closeSource': 'official',
 'closeTime': None,
 'companyName': 'Apple Inc',
 'currency': 'USD',
 'delayedPrice': None,
 'delayedPriceTime': None,
 'extendedChange': None,
 'extendedChangePercent': None,
 'extendedPrice': None,
 'extendedPriceTime': None,
 'high': None,
 'highSource': None,
 'highTime': None,
 'iexAskPrice': 145.24,
 'iexAskSize': 200,
 'iexBidPrice': 145.1,
 'iexBidSize': 121,
 'iexClose': 145.23,
 'iexCloseTime': 1666889640351,
 'iexLastUpdated': 1666889640351,
 'iexMarketPercent': 0.02644701521743253,
 'iexOpen': 148.1,
 'iexOpenTime': 1666877400166,
 'iexRealtimePrice': 145.23,
 'iexRealtimeSize': 75,
 'iexVolume': 1104483,
 'lastTradeTime': 1666889640351,
 'latestPrice': 145.23,
 'latestSource': 'IEX real time price',
 'latestTime': '12:54:00 PM',
 'latestUpdate': 1666889640351,
 'latestVolume': None,
 'low': None,
 'lowSource': 'IEX real time pr

In [110]:
# parsing the API call
price = data["latestPrice"]
market_cap = data["marketCap"]
print(market_cap/1000000000000)

2.33395531296


In [111]:
# adding stocks data to a pandas df
my_columns = ['Ticker', 'Stock Price', 'Market Capitalization', 'Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns = my_columns)
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy


In [112]:
final_dataframe.append(
    pd.Series(
        [
            symbol, # symbol = "AAPL"
            price, # price = data["latestPrice"]
            market_cap, # market_cap = data["marketCap"]
            'N/A'
        ],
        index = my_columns
    ),
    
    ignore_index=True
)

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,AAPL,145.23,2333955312960,N/A


In [113]:
# looping through the tickers in the list of stocks
final_dataframe = pd.DataFrame(columns = my_columns)
for stock in stocks['Ticker'][:5]:
    api_url = f"https://cloud.iexapis.com/stable/stock/{stock}/quote?token={IEX_CLOUD_API_TOKEN}"
    data = requests.get(api_url).json()
    final_dataframe = final_dataframe.append(
        pd.Series(
            [
                stock,
                data['latestPrice'],
                data['marketCap'],
                'N/A'
            ],
            index = my_columns
        ),
        ignore_index = True
    )
final_dataframe
    

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,136.825,40505750990,N/A
1,AAL,14.170,9209095569,N/A
2,AAP,188.430,11328079775,N/A
3,AAPL,145.230,2333955312960,N/A
4,ABBV,153.375,271181799921,N/A


In [114]:
# using batch API calls to improve performance

def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [122]:
symbol_groups = list(chunks(stocks["Ticker"], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(",".join(symbol_groups[i]))
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f"https://cloud.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}"
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(","):
        final_dataframe = final_dataframe.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]['quote']['latestPrice'],
                    data[symbol]['quote']['marketCap'],
                    'N/A'
                ],
                index = my_columns
        ),
        ignore_index = True
    )
final_dataframe

# had to remove 4 companies from the stock csv.

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,136.740,40480587542,N/A
1,AAL,14.170,9209095569,N/A
2,AAP,188.590,11337698693,N/A
3,AAPL,145.260,2334437435520,N/A
4,ABBV,153.290,271031511719,N/A
...,...,...,...,...
496,YUM,114.325,32530318340,N/A
497,ZBH,111.610,23417959752,N/A
498,ZBRA,279.030,14450947237,N/A
499,ZION,50.820,7646955278,N/A


In [120]:
# calculating the number of shares to buy

portfolio_size = input("Enter the value of your portfolio:")

try:
    val = float(portfolio_size)
except ValueError:
    print("That's not a number! \n Try again:")
    portfolio_size = input("Enter the value of your portfolio:")
    val = float(portfolio_size)

In [123]:
position_size = val/len(final_dataframe.index)
for i in range(0, len(final_dataframe["Ticker"]) -1):
    final_dataframe.loc[i, "Number of Shares to Buy"] = math.floor(position_size/final_dataframe["Stock Price"][i])
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,136.740,40480587542,14
1,AAL,14.170,9209095569,140
2,AAP,188.590,11337698693,10
3,AAPL,145.260,2334437435520,13
4,ABBV,153.290,271031511719,13
...,...,...,...,...
496,YUM,114.325,32530318340,17
497,ZBH,111.610,23417959752,17
498,ZBRA,279.030,14450947237,7
499,ZION,50.820,7646955278,39


In [124]:
# Formatting excel output

writer = pd.ExcelWriter('equal-weight-s&p500.xlsx', engine='xlsxwriter')
final_dataframe.to_excel(writer, sheet_name="equal-weight-sandp500", index = False)

In [125]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_format = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_format = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )


In [126]:
column_formats = { 
                    'A': ['Ticker', string_format],
                    'B': ['Price', dollar_format],
                    'C': ['Market Capitalization', dollar_format],
                    'D': ['Number of Shares to Buy', integer_format]
                    }

for column in column_formats.keys():
    writer.sheets['equal-weight-sandp500'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['equal-weight-sandp500'].write(f'{column}1', column_formats[column][0], string_format)

In [127]:
writer.save()